In [107]:
# type: ignore
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plot
%matplotlib inline

In [108]:
words = open('data/names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [109]:
len(words)

32033

In [110]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [111]:
# dataset creation
context_length = 3
# , labels
X, Y = [], []

for w in words:
    context = [0] * context_length
    
    for char in w + '.':
        ind = stoi[char]
        X.append(context)
        Y.append(ind)
        # print(''.join(itos[i] for i in context), '-->', itos[ind])
        context = context[1:] + [ind]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [112]:
X.shape, X.dtype, Y.shape, Y.dtype # type: ignore

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [113]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [114]:
# create 2-dimensional embeddings for each character
C = torch.randn((27, 2))
print(C.shape)
C

torch.Size([27, 2])


tensor([[ 0.4277,  0.9536],
        [-0.0627, -0.1185],
        [-2.6831,  0.8409],
        [-1.7305, -0.5252],
        [ 0.9636, -0.0167],
        [ 1.1850, -1.0321],
        [-0.5083,  0.6783],
        [-0.1542, -0.4464],
        [-1.0324,  0.3614],
        [-1.6597,  0.4516],
        [ 0.5170, -1.1308],
        [-0.6601, -0.1542],
        [ 0.7661, -0.3474],
        [ 1.3006, -0.2860],
        [ 0.1023, -0.9868],
        [ 1.9697,  1.3918],
        [ 0.5248,  0.3628],
        [-0.3170,  0.4321],
        [ 1.0551,  0.0145],
        [ 1.4077, -0.0753],
        [ 1.2235, -0.8573],
        [ 0.2043,  0.3955],
        [ 1.0340, -0.8872],
        [-0.9521, -2.0288],
        [-0.1429, -0.5658],
        [ 1.1650, -0.0664],
        [ 0.7382, -1.4797]])

In [115]:
# the embedding of 'd'
C[5]

tensor([ 1.1850, -1.0321])

In [116]:
# Note that this is equivalent to the above cell
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([ 1.1850, -1.0321])

In [117]:
# we can also index into C using the entire X tensor
# this has replaced each index in X with its embedding
emb = C[X]
C[X].shape

torch.Size([228146, 3, 2])

![Bengio et al. Architecture](images/bengioetal.png)

In [118]:
# first hidden layer
# 6 (each context piece has 3 embeddings of size 2) inputs to each of 100 (arbitrary) neurons
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [119]:
# emb.shape => (32, 3, 2) and W1 => (6, 100)
hidden_layer_one = emb.view(emb.shape[0], 6) @ W1 + b1
# tanh => [0, 1]
hidden_layer_one = torch.tanh(hidden_layer_one)
hidden_layer_one.shape

torch.Size([228146, 100])

In [120]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [121]:
# logits === final outputs
logits = hidden_layer_one @ W2 + b2
logits.shape

torch.Size([228146, 27])

In [122]:
parameters = [C, W1, b1, W2, b2]
sum(p.nelement() for p in parameters)

3481

### Softmax!

![Softmax in Neural Nets](images/softmax.png)

In [123]:
'''
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
# prob[torch.arange(32), Y] is the probability from each row that goes into the correct label y
loss = -prob[torch.arange(32), Y].log().mean()
'''
# built in function to perform the above operations
# prefer built in's over manual lines for baked in efficiency (for example not creating new tensors in memory) which makes the backwards passes faster
loss = F.cross_entropy(logits, Y)
loss

tensor(15.0199)

In [124]:
for p in parameters:
    p.requires_grad = True

### Training loop

In [125]:
for _ in range(1000):
    # forwards pass
    emb = C[X]
    
    hidden_layer_one = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

    logits = hidden_layer_one @ W2 + b2

    loss = F.cross_entropy(logits, Y)
    print(loss.item())
    
    # backwards pass
    for p in parameters:
        p.grad = None

    loss.backward()

    for p in parameters:
        p.data += -0.1 * p.grad

15.019881248474121
13.709667205810547
12.700278282165527
11.956734657287598
11.323923110961914
10.777948379516602
10.296542167663574
9.862217903137207
9.467780113220215
9.108856201171875
8.780131340026855
8.475985527038574
8.193107604980469
7.930363655090332


KeyboardInterrupt: 